# Deploy a Fine-tuned Korean ReRanker Model on AWS
 - 학습된 모델을 SageMaker를 이용하여 배포합니다.

## AutoReload

In [ ]:
%load_ext autoreload
%autoreload 2

## 1. Inference by local

In [ ]:
import os
import torch
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
model_dir = "./model"
model_data_uri = "s3://sagemaker-us-east-1-419974056037/fine-tune-reranker-kr/training/model-output/huggingface-pytorch-training-2023-12-21-07-22-38-356/output/model.tar.gz"
#model_data_uri = "s3://sagemaker-us-east-1-419974056037/fine-tune-reranker-kr/training/checkpoints/g3_b1_gas32/checkpoint-10000/"
if "model.tar.gz" in model_data_uri:
    model_data_uri = model_data_uri.replace("model.tar.gz", "")
model_tar_data = os.path.join(model_dir, "model.tar.gz")
model_data = os.path.join(model_dir, "model_data")

### 1.1 Model source 선택
- `hf`: 허깅페이스에 등록된 모델
- `train`: SageMaker를 통해 학습한 모델
- `train_ckpt`: SageMaker를 통해 학습하는 과정에서 발생되는 checkpoint 모델

In [ ]:
model_source = "train" #["hf", "train", "train_ckpt"]

In [ ]:
if model_source != "hf":
    !rm -rf $model_dir
    !mkdir $model_dir
    !aws s3 sync $model_data_uri $model_dir
    if model_source == "train":
        !mkdir $model_data
        !tar -xvf $model_tar_data -C $model_data

In [ ]:
if model_source == "hf": model_path = "Dongjin-kr/ko-reranker" #model_path = "BAAI/bge-reranker-large"
elif model_source == "train": model_path = model_data
elif model_source == "train_ckpt": model_path = model_dir

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

model.eval()

In [ ]:
def exp_normalize(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()

* [Tip] 스코어는 두 문장이 관련이 있을 수록 1에 가까워 진다

In [ ]:
pairs = [["나는 너를 싫어해", "나는 너를 사랑해"],
         ["나는 너를 좋아해", "너에 대한 나의 감정은 사랑 일 수도 있어"]]

with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    scores = exp_normalize(scores.numpy())
    print (f'first: {scores[0]}, second: {scores[1]}')

## 2. Inference by cloud

### 2.1 Deploy model on AWS

In [ ]:
import json
import boto3
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

In [ ]:
%%writefile $model_data/

In [ ]:
model_data

#### [주의] Building serving image
- Fine-tuned reranker 모델 서빙은 AWS의 `HuggingFace Inference Containers` 를 사용합니다. 
    - Native Deep Learning Conatiner (DLC)의 정보는 [link](https://github.com/aws/deep-learning-containers/blob/master/available_images.md)를 통해 확인하세요.
- 원할한 서빙을 위해서는 `transformer >= 4.36.2` 가 필요합니다. (transformer ver.: 4.28.1 in native container)
- 때문에 해당 예제에서는 native container image를 기반으로 transformers version 4.36.2로 업데이트 하여 serving 하도록 합니다. 

#### Create model_data for deploy
> 이 단계는 약 3분 소요 됩니다.

In [ ]:
model_data_path = "s3://sagemaker-us-east-1-419974056037/fine-tune-reranker-kr/training/checkpoints/g3_b2_gas8/checkpoint-15000/"
#model_data_path = "s3://sagemaker-us-east-1-419974056037/fine-tune-reranker-kr/training/model-output/huggingface-pytorch-training-2023-12-21-07-22-38-356/output/model.tar.gz"

is_ckpt = False
if "model.tar.gz" not in model_data_path: is_ckpt = True
if not is_ckpt: model_data_path = model_data_path.replace("model.tar.gz", "")

model_tmp_dir = "./model"
model_data_tmp_uri = "s3://sagemaker-us-east-1-419974056037/fine-tune-reranker-kr/training/model_data_for_depoly"

!rm -rf $model_tmp_dir
!mkdir $model_tmp_dir
!mkdir $model_tmp_dir/code
!echo transformers==4.36.2 > $model_tmp_dir/code/requirements.txt
!aws s3 sync $model_data_path $model_tmp_dir

if not is_ckpt:
    !tar -zxvf $model_tmp_dir/model.tar.gz -C $model_tmp_dir/ 
    !rm -rf $model_tmp_dir/model.tar.gz

!tar -zcvf ./model.tar.gz -C $model_tmp_dir .
!aws s3 cp ./model.tar.gz $model_data_tmp_uri/model.tar.gz
!rm -rf ./model.tar.gz

model_data_path = os.path.join(model_data_tmp_uri, "model.tar.gz")

    

print (f'model_data_path: {model_data_path}')

#### Depoly model on cloud

In [ ]:
depoly = True

In [ ]:
if depoly:

    print (f'model deloy from : {model_data_path}')

    try:
        role = sagemaker.get_execution_role()
    except ValueError:
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

    # Hub Model configuration. https://huggingface.co/models
    hub = {
        'HF_MODEL_ID':'Dongjin-kr/ko-reranker',
        'HF_TASK':'text-classification'
    }

    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
        model_data=model_data_path,
        transformers_version='4.28.1',
        pytorch_version='2.0.0',
        py_version='py310',
        #env=hub,
        role=role,
        #image_uri=ecr_repository_uri
    )

    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
        initial_instance_count=1,
        instance_type='ml.g5.xlarge',
        wait=True
    )

    print(f'Accept: {predictor.accept}')
    print(f'ContentType: {predictor.content_type}')
    print(f'Endpoint: {predictor.endpoint}')

### 2.2 Invocation

In [ ]:
runtime_client = boto3.Session().client('sagemaker-runtime')
print (f'runtime_client: {runtime_client}')

#### [중요] 아래 endpoint_name 변수를 Depoly model on cloud 의 실행 결과인 Endpoint 값으로 바꾸어 주세요. 
- example: endpoint_name = "huggingface-pytorch-inference-2024-01-03-05-12-12-769"  


In [ ]:
endpoint_name = "<Type Endpoint Name>"  
deserializer = "application/json"

In [ ]:
payload = json.dumps(
    {
        "inputs": [
            {"text": "나는 너를 싫어해", "text_pair": "나는 너를 사랑해"},
            {"text": "나는 너를 좋아해", "text_pair": "너에 대한 나의 감정은 사랑 일 수도 있어"}
        ]
    }
)

In [ ]:
%%time
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept=deserializer,
    Body=payload
)
## deserialization
out = json.loads(response['Body'].read().decode()) ## for json
print (f'Response: {out}')

## 3. Comparison before/after finetuning

In [ ]:
import numpy as np

In [ ]:
pairs = [
    ["섬유소 용해제 또는 혈전 용해제의 작용 메커니즘은 무엇입니까?", \
     "Bailliãre의 임상 혈액학. 6 혈전 용해제의 작용 메커니즘. 6 혈전 용해제의 작용 메커니즘 JEFFREY I. WEITZ 피브린은 지혈, 염증 또는 조직 복구 과정에서 형성되는 일시적인 역할을 하며 정상적인 조직 기능 및 구조를 복원하려면 분해되어야 합니다."
    ],
    ["섬유소 용해제 또는 혈전 용해제의 작용 메커니즘은 무엇입니까?", \
     "단백질 분해 효소는 점막의 팽창 감소, 모세 혈관 투과성 감소, 혈전 형성 섬유소 침전물 및 미세 혈전 용해 등 다양한 메커니즘에 의해 염증 과정을 조절합니다.효소는 혈액의 점도 (두께) 를 줄임으로써 혈액 순환을 개선합니다. 브로멜라인, 파파인, 판크레아틴, 트립신, 키모트립신, 루틴과 같은 단백질 분해 효소는 염증 반응의 필수 조절제 및 조절제입니다.이들의 중요한 작용 중에는 대식세포의 '식욕'이 7~10배 증가하고 자연 살해 (NK) 세포의 효능이 7~10배 증가한다는 것입니다."
    ]
]
payload = json.dumps(
    {
        "inputs": [
            {"text": pairs[0][0], "text_pair": pairs[0][1]},
            {"text": pairs[1][0], "text_pair": pairs[1][1]}
        ]
    }
)

with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    scores = exp_normalize(scores.numpy())
    print(f'original: {scores}')

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept=deserializer,
    Body=payload
)
## deserialization
out = json.loads(response['Body'].read().decode()) ## for json
print (f'Fine-tune: {out}')

## [Optional] Upload to HuggingFace

In [ ]:
# from huggingface_hub import login
# login()

In [ ]:
# hf_hub_path = "Dongjin-kr/ko-reranker"
# print(hf_hub_path)

In [ ]:
# model.push_to_hub(
#     repo_id=hf_hub_path,
#     safe_serialization=False
# )
# tokenizer.push_to_hub(hf_hub_path, legacy_format='False')